In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
        
    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i']) 
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)

    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i'])
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = 523cbb7
molecule = {'co2': 0.0008}
band = [11]
atmpro = mls
tsfc = 294


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o co2 band9
  atmpro = mls
  band = 9
  commitnumber = a06b618
  conc = 0.0004
  dv = 0.001
  klin = 6.5e-24
  molecule = co2
  ng_adju = [0, 0]
  ng_refs = [3, 3]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (50, 250)]
  tsfc = 294
  vmax = 3000
  vmin = 1900
  w_diffuse = [(1.66, 1.66, 1.75), (1.75, 1.6, 1.85)]
  wgt = [(0.7, 0.8, 0.7), (0.8, 0.7, 0.8)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1  0.180856                    1  0.428764
0.000750        2  0.015722                    2  0.044931
0.001052        3  0.013947                    3  0.005200
0.001476        4  0.008595                    4 -0.011757
0.002070        5  0.002623                    5 -0.021366
0.002904        6 -0.003201                    6 -0.020984
0.004074        7 -0.009094                    7 -0.021932
0.005714        8 -0.015228                    8 -0.019483
0.008015        9 -0.021511                    9 -0.020046
0.011243       10 -0.027870                   10 -0.020038
0.015771       11 -0.034294                   11 -0.022585
0.022122       12 -0.040994                   12 -0.028193
0.031031       13 -0.048820                   13 -0.037421
0.043528       14 -0.062120                   14 -0.055808
0.061057       15 -0.078013                   15 -0.075402
0.085645       16 -0.083801                   16 -0.075957
0.120136       17 -0.078824                   17 -0.057892
0.168516       18 -0.072447                   18 -0.041446
0.236378       19 -0.058150                   19 -0.029139
0.331549       20 -0.029119                   20 -0.012905
0.465100       21  0.026567                   21  0.027779
0.652400       22  0.155155                   22  0.147814
0.915100       23  0.360708                   23  0.346954
1.283650       24  0.360813                   24  0.376318
1.800600       25  0.202114                   25  0.257554
2.525700       26  0.108382                   26  0.151521
3.542800       27  0.057817                   27  0.066053
4.969550       28  0.028726                   28  0.015172
6.970850       29  0.013442                   29  0.006420
9.778100       30  0.005458                   30  0.011293
13.715850      31  0.000599                   31  0.008056
19.239350      32 -0.001974                   32  0.001199
26.987250      33 -0.001919                   33 -0.002384
37.855300      34 -0.001221                   34 -0.001830
53.100050      35 -0.001376                   35 -0.001276
73.887500      36 -0.001921                   36 -0.001739
97.662500      37 -0.002207                   37 -0.002015
121.437500     38 -0.002334                   38 -0.002158
145.212500     39 -0.002918                   39 -0.002837
168.987500     40 -0.003865                   40 -0.003892
192.762500     41 -0.004395                   41 -0.004455
216.537500     42 -0.004221                   42 -0.004314
240.312500     43 -0.004194                   43 -0.004286
264.087500     44 -0.004266                   44 -0.004340
287.862500     45 -0.004319                   45 -0.004291
311.637500     46 -0.004337                   46 -0.004237
335.412500     47 -0.004309                   47 -0.003996
359.187500     48 -0.004231                   48 -0.003782
382.962500     49 -0.004096                   49 -0.003478
406.737500     50 -0.003899                   50 -0.003112
430.512500     51 -0.003654                   51 -0.002697
454.287500     52 -0.003388                   52 -0.002400
478.062500     53 -0.003125                   53 -0.001989
501.837500     54 -0.002861                   54 -0.001726
525.612500     55 -0.002575                   55 -0.001469
549.387500     56 -0.002253                   56 -0.001211
573.162500     57 -0.001889                   57 -0.000937
596.937500     58 -0.001481                   58 -0.000638
620.712500     59 -0.001028                   59 -0.000303
644.487500     60 -0.000530                   60  0.000071
668.262500     61  0.000017                   61  0.000526
692.037500     62  0.000620                   62  0.000827
715.812500     63  0.001291                   63  0.001487
739.587500     64  0.002048                   64  0.002188
763.362500     65  0.002927                   65  0.002880
78

# Fluxes by Layer

CRD                        CLIRAD                \
                       flug      fldg     fnetg      flug          fldg   
pressure    level                                                         
0.000000    1     -5.320607  0.000000 -5.320607 -5.242120  8.381454e-09   
0.000624    2     -5.320613  0.000019 -5.320594 -5.242127  3.870245e-05   
0.000876    3     -5.320615  0.000022 -5.320593 -5.242131  4.341266e-05   
0.001229    4     -5.320619  0.000026 -5.320593 -5.242135  4.807412e-05   
0.001723    5     -5.320623  0.000031 -5.320592 -5.242141  5.309169e-05   
0.002417    6     -5.320629  0.000037 -5.320592 -5.242148  5.889467e-05   
0.003391    7     -5.320637  0.000045 -5.320593 -5.242158  6.634493e-05   
0.004757    8     -5.320648  0.000054 -5.320594 -5.242172  7.609202e-05   
0.006672    9     -5.320662  0.000065 -5.320597 -5.242189  8.941954e-05   
0.009359    10    -5.320683  0.000078 -5.320604 -5.242214  1.074306e-04   
0.013128    11    -5.320711  0.000094 -5.320617 -5.242247  1.322006e-04   
0.018415    12    -5.320751  0.000113 -5.320638 -5.242295  1.657194e-04   
0.025830    13    -5.320808  0.000134 -5.320674 -5.242364  2.102667e-04   
0.036232    14    -5.320894  0.000159 -5.320734 -5.242468  2.681686e-04   
0.050823    15    -5.321032  0.000190 -5.320842 -5.242637  3.405693e-04   
0.071291    16    -5.321271  0.000240 -5.321031 -5.242924  4.441877e-04   
0.100000    17    -5.321676  0.000361 -5.321316 -5.243385  6.467532e-04   
0.140271    18    -5.322324  0.000632 -5.321692 -5.244078  1.063732e-03   
0.196760    19    -5.323340  0.001163 -5.322177 -5.245102  1.810770e-03   
0.275997    20    -5.324901  0.002179 -5.322723 -5.246593  3.028132e-03   
0.387100    21    -5.327210  0.004104 -5.323106 -5.248691  4.956553e-03   
0.543100    22    -5.330344  0.007730 -5.322615 -5.251383  8.161223e-03   
0.761700    23    -5.333080  0.014482 -5.318597 -5.253342  1.394846e-02   
1.068500    24    -5.330774  0.025286 -5.305488 -5.250463  2.367971e-02   
1.498800    25    -5.322437  0.035341 -5.287097 -5.242312  3.471294e-02   
2.102400    26    -5.312796  0.040151 -5.272645 -5.232851  4.366894e-02   
2.949000    27    -5.303544  0.041768 -5.261776 -5.223467  4.948204e-02   
4.136600    28    -5.295107  0.041465 -5.253642 -5.214494  4.980193e-02   
5.802500    29    -5.287720  0.039746 -5.247974 -5.206250  4.455287e-02   
8.139200    30    -5.281286  0.037033 -5.244253 -5.198718  3.879780e-02   
11.417000   31    -5.275621  0.033488 -5.242133 -5.191862  3.632734e-02   
16.014700   32    -5.270937  0.029129 -5.241807 -5.185980  3.483370e-02   
22.464000   33    -5.267861  0.024546 -5.243315 -5.181698  3.146738e-02   
31.510500   34    -5.266403  0.021031 -5.245371 -5.179039  2.625318e-02   
44.200100   35    -5.266115  0.018908 -5.247206 -5.177483  2.194641e-02   
62.000000   36    -5.267496  0.017389 -5.250107 -5.177603  1.937596e-02   
85.775000   37    -5.271808  0.016289 -5.255518 -5.181039  1.791513e-02   
109.550000  38    -5.277849  0.016115 -5.261735 -5.186682  1.788275e-02   
133.325000  39    -5.285002  0.016693 -5.268309 -5.193723  1.884577e-02   
157.100000  40    -5.293877  0.017348 -5.276529 -5.202793  1.992620e-02   
180.875000  41    -5.305983  0.018570 -5.287413 -5.215536  2.170545e-02   
204.650000  42    -5.321864  0.022072 -5.299792 -5.232385  2.600557e-02   
228.425000  43    -5.340508  0.028829 -5.311679 -5.252228  3.369821e-02   
252.200000  44    -5.361546  0.038054 -5.323491 -5.274546  4.394478e-02   
275.975000  45    -5.384941  0.049437 -5.335504 -5.299339  5.651402e-02   
299.750000  46    -5.410672  0.063003 -5.347669 -5.326524  7.161239e-02   
323.525000  47    -5.438702  0.078819 -5.359882 -5.356157  8.931298e-02   
347.300000  48    -5.468971  0.096953 -5.372018 -5.388034  1.099332e-01   
371.075000  49    -5.501395  0.117461 -5.383934 -5.422145  1.333930e-01   
394.850000  50    -5.535846  0.140375 -5.395471 -5.458339  1.597917e-01   
418.625000  51    -5.572135  0.165684 -5.406452 -

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')